In [ ]:
import json
import numpy as np
from collections import defaultdict

In [ ]:
league = json.load(open('fbgm_fa.json','rb'))
season = 2021
for k in league['gameAttributes']:
    if k['key'] == 'season':
        season = k['value']

In [ ]:
# for the form exp(x[0]*i)*x[1]
# fit to FBGM team rating coeffcients 
# tweaks 0.25*QB1 for Backup QB, LB smoothed out by hand, backup K and P set to min weight of set
pos_V = {'QB': [-1.396, 0.275],
 'RB': [-1.49, 0.019],
 'TE': [-1.527, 0.031],
 'WR': [-0.524, 0.061],
 'OL': [-0.213, 0.181],
 'CB': [-0.568, 0.136],
 'S': [-0.508, 0.101],
 'LB': [-0.389, 0.072],
 'DL': [-0.286, 0.234],
 'K': [-4.386, 0.08],
 'P': [-3.936, 0.051]}


In [ ]:
pool = [] # this is the set of draftees OR FA pool as well

teams = defaultdict(lambda: {k:[] for k in pos_V})

In [ ]:
for p in league['players']:
    # use VALUE instead of OVR to capture age+pot blend
    pos = p['ratings'][0]['pos']
    val = p['valueFuzz']
    pid = p['pid']

    # value is scaled for some positions
    # scale back!
    if pos == 'QB':
        val /= 1.25
    if pos == 'K' or pos == 'P':
        val /= 0.25
    
    ppair = (pos,val,pid)

    if p['tid'] >= 0 :
        teams[p['tid']][ppair[0]].append(ppair[1])
    # DRAFT
    #elif p['draft']['year'] == season:
    #    pool.append(ppair)
    # FA
    elif p['tid'] == -1:
        pool.append(ppair)

In [ ]:
def fbgm_value(team,weights):
    teamvt = [v*np.exp(weights[0]*i)*weights[1] for i,v in enumerate(sorted(team,reverse=True))]
    return sum(teamvt)

In [ ]:
team_drafts = {}
for tid,team in teams.items():
    player_values = []
    for pos,value,pid in pool:
        pre_value = fbgm_value(team[pos], pos_V[pos])
        post_value = fbgm_value(team[pos] + [value], pos_V[pos])
        value_add = post_value-pre_value
        player_values.append((value_add,pid,pos,value))
    team_drafts[tid] = sorted(player_values,reverse=True)

In [ ]:
team_drafts[0]

In [ ]:
team_drafts[1]